# ALS
https://github.com/microsoft/recommenders/blob/main/examples/03_evaluate/als_movielens_diversity_metrics.ipynb

In [1]:
%load_ext autoreload
%autoreload 2

import sys

import pyspark
from pyspark.ml.recommendation import ALS
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, FloatType, IntegerType, LongType, StructType, StructField
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover
from pyspark.ml.feature import HashingTF, CountVectorizer, VectorAssembler

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.utils.notebook_utils import is_jupyter
from recommenders.datasets.spark_splitters import spark_random_split
from recommenders.evaluation.spark_evaluation import SparkRatingEvaluation, SparkRankingEvaluation, SparkDiversityEvaluation
from recommenders.utils.spark_utils import start_or_get_spark

from pyspark.sql.window import Window
import pyspark.sql.functions as F

import numpy as np
import pandas as pd

print("System version: {}".format(sys.version))
print("Spark version: {}".format(pyspark.__version__))

System version: 3.7.0 (default, Oct  9 2018, 10:31:47) 
[GCC 7.3.0]
Spark version: 2.4.8


## Set Paramters for Dataset

In [2]:
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

# user, item column names
COL_USER="UserId"
COL_ITEM="MovieId"
COL_RATING="Rating"

## Spark environment and Dataset setup

In [3]:
spark = start_or_get_spark("ALS PySpark", memory="16g")
spark

Setting cross-join to enabled, because pyspark has ways to disable it otherwise

In [41]:
spark.conf.set("spark.sql.crossJoin.enabled", "true")

Creating column types/names with struct and then loading the built in movielens

In [4]:
# Note: The DataFrame-based API for ALS currently only supports integers for user and item ids.
schema = StructType(
    (
        StructField(COL_USER, IntegerType()),
        StructField(COL_ITEM, IntegerType()),
        StructField(COL_RATING, FloatType()),
        StructField("Timestamp", LongType()),
    )
)

In [5]:
data = movielens.load_spark_df(spark, size=MOVIELENS_DATA_SIZE, schema=schema, title_col="title", genres_col="genres")
print(data.schema)
data.show()

100%|██████████| 4.81k/4.81k [00:00<00:00, 19.1kKB/s]


StructType(List(StructField(MovieId,IntegerType,true),StructField(UserId,IntegerType,true),StructField(Rating,FloatType,true),StructField(Timestamp,LongType,true),StructField(title,StringType,true),StructField(genres,StringType,true)))
+-------+------+------+---------+--------------------+------+
|MovieId|UserId|Rating|Timestamp|               title|genres|
+-------+------+------+---------+--------------------+------+
|     26|   138|   5.0|879024232|Brothers McMullen...|Comedy|
|     26|   224|   3.0|888104153|Brothers McMullen...|Comedy|
|     26|    18|   4.0|880129731|Brothers McMullen...|Comedy|
|     26|   222|   3.0|878183043|Brothers McMullen...|Comedy|
|     26|    43|   5.0|883954901|Brothers McMullen...|Comedy|
|     26|   201|   4.0|884111927|Brothers McMullen...|Comedy|
|     26|   299|   4.0|878192601|Brothers McMullen...|Comedy|
|     26|    95|   3.0|880571951|Brothers McMullen...|Comedy|
|     26|    89|   3.0|879459909|Brothers McMullen...|Comedy|
|     26|   361|   3

split data, ratio argument indicates the ratio of the training data.

In [6]:
train_df, test_df = spark_random_split(data.select(COL_USER, COL_ITEM, COL_RATING), ratio=0.75, seed=123)
print ("N train_df", train_df.cache().count())
train_df.show()
print ("N test_df", test_df.cache().count())

N train_df 75066
+------+-------+------+
|UserId|MovieId|Rating|
+------+-------+------+
|     1|     26|   3.0|
|     7|     29|   3.0|
|     7|    474|   5.0|
|    10|    474|   4.0|
|    11|     29|   3.0|
|    13|    474|   4.0|
|    18|     26|   4.0|
|    18|    474|   4.0|
|    18|    964|   3.0|
|    22|     29|   1.0|
|    25|    474|   4.0|
|    41|    474|   5.0|
|    43|     26|   5.0|
|    44|    474|   4.0|
|    56|     29|   3.0|
|    58|    474|   4.0|
|    59|    474|   5.0|
|    60|    474|   5.0|
|    73|    474|   5.0|
|    76|    474|   5.0|
+------+-------+------+
only showing top 20 rows

N test_df 24934


In [45]:
print(data.cache().count())

1000209


Check average number of ratings per user.

In [46]:
rowsPerUser = data.groupBy("UserId").count().sort('count', ascending = False)
rowsPerUser.show()
print(rowsPerUser.cache().count())
rowsPerUser.agg({'count': 'mean'}).show()
rowsPerUser.agg({'count': 'min'}).show()

+------+-----+
|UserId|count|
+------+-----+
|  4169| 2314|
|  1680| 1850|
|  4277| 1743|
|  1941| 1595|
|  1181| 1521|
|   889| 1518|
|  3618| 1344|
|  2063| 1323|
|  1150| 1302|
|  1015| 1286|
|  5795| 1277|
|  4344| 1271|
|  1980| 1260|
|  2909| 1258|
|  1449| 1243|
|  4510| 1240|
|   424| 1226|
|  4227| 1222|
|  5831| 1220|
|  3841| 1216|
+------+-----+
only showing top 20 rows

6040
+-----------------+
|       avg(count)|
+-----------------+
|165.5975165562914|
+-----------------+

+----------+
|min(count)|
+----------+
|        20|
+----------+



## Use cross join to create all possible user-item pairs
Use training data

In [7]:
users = train_df.select(COL_USER).distinct()
items = train_df.select(COL_ITEM).distinct()
user_item = users.crossJoin(items)

In [48]:
print(users.count())
print(items.count())
print(user_item.count())
user_item.show()

6040
3670
22166800
+------+-------+
|UserId|MovieId|
+------+-------+
|   148|    463|
|   148|   1591|
|   148|   3918|
|   148|   3175|
|   148|    496|
|   148|   1238|
|   148|   3794|
|   148|   1342|
|   148|    833|
|   148|   2366|
|   148|   1829|
|   148|    471|
|   148|   1959|
|   148|   1580|
|   148|   2142|
|   148|   2659|
|   148|   2866|
|   148|   1088|
|   148|   2122|
|   148|   1645|
+------+-------+
only showing top 20 rows



## Train Model and get top K recommendations

als model

In [8]:
header = {
    "userCol": COL_USER,
    "itemCol": COL_ITEM,
    "ratingCol": COL_RATING,
}


##could modify and have another algo here
als = ALS(
    rank=10,
    maxIter=15,
    implicitPrefs=False,
    regParam=0.05,
    coldStartStrategy='drop',
    nonnegative=False,
    seed=42,
    **header
)

training

In [9]:
with Timer() as train_time:
    model = als.fit(train_df)

print("Took {} seconds for training.".format(train_time.interval))

Took 2.9477865416556597 seconds for training.


recommend all movies to all users and then takeout the ones already in training

In [10]:
# Score all user-item pairs
dfs_pred = model.transform(user_item)
dfs_pred.show()

+------+-------+----------+
|UserId|MovieId|prediction|
+------+-------+----------+
|   148|    148| 3.0596461|
|   463|    148|   2.92153|
|   471|    148| 3.4413671|
|   496|    148| 2.3569374|
|   833|    148| 1.5069426|
|   243|    148| 2.3693802|
|   392|    148| 2.2736673|
|   540|    148|   3.12988|
|   623|    148| 3.0082514|
|   737|    148| 1.2959054|
|   858|    148| 2.2418387|
|   897|    148| 3.7995431|
|    31|    148| 1.3679194|
|   516|    148| 3.5672064|
|    85|    148|  2.805176|
|   137|    148|  4.088493|
|   251|    148| 2.8925745|
|   451|    148| 3.3047304|
|   580|    148| 3.4140422|
|   808|    148| 3.5600035|
+------+-------+----------+
only showing top 20 rows



In [11]:
# Remove seen items - Remember we only used training data to create user_item
dfs_pred_exclude_train = dfs_pred.alias("pred").join(
    train_df.alias("train"),
    (dfs_pred[COL_USER] == train_df[COL_USER]) & (dfs_pred[COL_ITEM] == train_df[COL_ITEM]),
    how='outer'
)

top_all = dfs_pred_exclude_train.filter(dfs_pred_exclude_train["train.Rating"].isNull()) \
    .select('pred.' + COL_USER, 'pred.' + COL_ITEM, 'pred.' + "prediction")

print(top_all.count())

#top_all.show()

1464853


In [12]:
window = Window.partitionBy(COL_USER).orderBy(F.col("prediction").desc())
top_k_reco = top_all.select("*", F.row_number().over(window).alias("rank")).filter(F.col("rank") <= TOP_K).drop("rank")
 
print(top_k_reco.count())
top_k_reco.show()

9430
+------+-------+----------+
|UserId|MovieId|prediction|
+------+-------+----------+
|   148|    919| 6.3261137|
|   148|    353| 6.2676616|
|   148|    968| 6.1783676|
|   148|    253| 6.1438556|
|   148|   1512| 5.9449964|
|   148|   1227| 5.8750854|
|   148|   1251| 5.6491814|
|   148|    115|   5.64693|
|   148|    244|  5.636389|
|   148|     57| 5.4875755|
|   463|   1167|  5.515826|
|   463|    958|  5.457613|
|   463|    697|  5.267504|
|   463|    242|  5.133097|
|   463|   1036| 5.0543756|
|   463|    408| 4.9970107|
|   463|    601|  4.986263|
|   463|    899|  4.964865|
|   463|     57|  4.882764|
|   463|    613|  4.845562|
+------+-------+----------+
only showing top 20 rows



## Make topk reco with examples

In [13]:
import fun

In [14]:
top_k_pd = fun.movielens_to_pandas(top_k_reco)
top_k_pd.head()

,UserId,MovieId,prediction
0,148,919,6.326114
1,148,353,6.267662
2,148,968,6.178368
3,148,253,6.143856
4,148,1512,5.944996


In [15]:
data_pd = fun.movielens_to_pandas(data)
data_pd.head()

,MovieId,UserId,Rating,Timestamp,title,genres
0,26,138,5.0,879024232,"Brothers McMullen, The (1995)",Comedy
1,26,224,3.0,888104153,"Brothers McMullen, The (1995)",Comedy
2,26,18,4.0,880129731,"Brothers McMullen, The (1995)",Comedy
3,26,222,3.0,878183043,"Brothers McMullen, The (1995)",Comedy
4,26,43,5.0,883954901,"Brothers McMullen, The (1995)",Comedy


In [16]:
top_k_with_info = (top_k_pd.join(data_pd[['MovieId', 'title', 'genres']].drop_duplicates().set_index('MovieId'), 
                                on='MovieId', 
                                how='inner').sort_values(by=['UserId', 'prediction'], ascending=False))
display(top_k_with_info.head(30))

,UserId,MovieId,prediction,title,genres
7310,943,320,5.774078,Paradise Lost: The Child Murders at Robin Hood...,Documentary
7311,943,962,5.159168,Ruby in Paradise (1993),Drama
7312,943,1131,5.127838,Safe (1995),Thriller
7313,943,1134,5.078665,Get on the Bus (1996),Drama
7314,943,127,5.057547,"Godfather, The (1972)",Action|Crime|Drama
7315,943,219,4.975318,"Nightmare on Elm Street, A (1984)",Horror
7316,943,963,4.927910,Some Folks Call It a Sling Blade (1993),Drama|Thriller
7317,943,1664,4.924054,8 Heads in a Duffel Bag (1997),Comedy
7318,943,641,4.920611,Paths of Glory (1957),Drama|War
7319,943,74,4.916291,Faster Pussycat! Kill! Kill! (1965),Action|Comedy|Drama


In [17]:
top_k_with_info.to_csv("als_topk_visual_results.csv", index = False)

## Metric functions

In [17]:
##different diversity metrics parsed form recall eval....object?
def get_ranking_results(ranking_eval):
    metrics = {
        "Precision@k": ranking_eval.precision_at_k(),
        "Recall@k": ranking_eval.recall_at_k(),
        "NDCG@k": ranking_eval.ndcg_at_k(),
        "Mean average precision": ranking_eval.map_at_k()
      
    }
    return metrics  

##different diversity metrics parsed form diversity eval....object?
def get_diversity_results(diversity_eval):
    metrics = {
        "catalog_coverage":diversity_eval.catalog_coverage(),
        "distributional_coverage":diversity_eval.distributional_coverage(), 
        "novelty": diversity_eval.novelty(), 
        "diversity": diversity_eval.diversity(), 
        "serendipity": diversity_eval.serendipity()
    }
    return metrics


def get_rating_results(rating_eval):
    metrics = {
     'rmse': rating_eval.rmse(),
     'mean absolute error' : rating_eval.mae(),
     'R squared': rating_eval.rsquared(),
     'explained variance': rating_eval.exp_var()
    }
    return metrics

def generate_summary(data, algo, k, ranking_metrics, diversity_metrics):
    summary = {"Data": data, "Algo": algo, "K": k}

    if ranking_metrics is None:
        ranking_metrics = {           
            "Precision@k": np.nan,
            "Recall@k": np.nan,            
            "nDCG@k": np.nan,
            "MAP": np.nan,
        }
        #update just adds to the back of the dictionary.
    summary.update(ranking_metrics)
    summary.update(diversity_metrics)
    return summary

## ALS Ranking results

In [18]:
als_ranking_eval = SparkRankingEvaluation(
    test_df, 
    top_all, 
    k = TOP_K, 
    col_user="UserId", 
    col_item="MovieId",
    col_rating="Rating", 
    col_prediction="prediction",
    relevancy_method="top_k"
)

als_ranking_metrics = get_ranking_results(als_ranking_eval)

In [19]:
cols = [ "Precision@k", "Recall@k", "NDCG@k", "Mean average precision"]
ranking_results = pd.DataFrame(columns=cols)

ranking_results.loc[1] = als_ranking_metrics

ranking_results.head()

,Precision@k,Recall@k,NDCG@k,Mean average precision
1,0.053287,0.038843,0.052833,0.007391


## Diversity results

In [20]:
als_diversity_eval = SparkDiversityEvaluation(
    train_df = train_df, 
    reco_df = top_k_reco,
    col_user = COL_USER, 
    col_item = COL_ITEM
)

als_diversity_metrics = get_diversity_results(als_diversity_eval)

In [21]:
cols = ["catalog_coverage", "distributional_coverage","novelty", "diversity", "serendipity"]
diversity_results = pd.DataFrame(columns=cols)

diversity_results.loc[1] = als_diversity_metrics

diversity_results.head()

,catalog_coverage,distributional_coverage,novelty,diversity,serendipity
1,0.515003,8.497723,11.391092,0.880526,0.868944


## Combined results

In [22]:
als_results = generate_summary(MOVIELENS_DATA_SIZE, "als", TOP_K, als_ranking_metrics, als_diversity_metrics)
cols = ["Data", "Algo", "K", "Precision@k", "Recall@k", "NDCG@k", "Mean average precision","catalog_coverage", "distributional_coverage","novelty", "diversity", "serendipity" ]
summary_results = pd.DataFrame(columns = cols)
summary_results.loc[1] = als_results
summary_results.head()

,Data,Algo,K,Precision@k,Recall@k,NDCG@k,Mean average precision,catalog_coverage,distributional_coverage,novelty,diversity,serendipity
1,100k,als,20,0.053287,0.038843,0.052833,0.007391,0.515003,8.497723,11.391092,0.880526,0.868944


## Add rating evaluation

In [23]:
als_rating_eval = SparkRatingEvaluation(
    test_df, 
    top_all,  
    col_user="UserId", 
    col_item="MovieId",
    col_rating="Rating", 
    col_prediction="prediction")

rating_results = get_rating_results(als_rating_eval)

In [24]:
cols = ['rmse', 'mean absolute error', 'R squared','explained variance']

rating_res = pd.DataFrame(columns=cols)
rating_res.loc[1] = rating_results
rating_res.head()

,rmse,mean absolute error,R squared,explained variance
1,0.971761,0.757398,0.254696,0.259558


## Content based version

### convert text input into feature vectors

In [25]:
# Get movie features "title" and "genres"
movies = (
    data.groupBy("MovieId", "title", "genres").count()
    .na.drop()  # remove rows with null values
    .withColumn("genres", F.split(F.col("genres"), "\|"))  # convert to array of genres
    .withColumn("title", F.regexp_replace(F.col("title"), "[\(),:^0-9]", ""))  # remove year from title
    .drop("count")  # remove unused columns
)
movies.show(5)

+-------+--------------------+---------+
|MovieId|               title|   genres|
+-------+--------------------+---------+
|    167|   Private Benjamin | [Comedy]|
|   1343|         Lotto Land |  [Drama]|
|   1607|  Hurricane Streets |  [Drama]|
|    966|Affair to Remembe...|[Romance]|
|      9|   Dead Man Walking |  [Drama]|
+-------+--------------------+---------+
only showing top 5 rows



## tokenize title and remove stop words

In [26]:
# tokenize "title" column
title_tokenizer = Tokenizer(inputCol="title", outputCol="title_words")
tokenized_data = title_tokenizer.transform(movies)
tokenized_data.show(10)

+-------+--------------------+--------------------+--------------------+
|MovieId|               title|              genres|         title_words|
+-------+--------------------+--------------------+--------------------+
|    167|   Private Benjamin |            [Comedy]| [private, benjamin]|
|   1343|         Lotto Land |             [Drama]|       [lotto, land]|
|   1607|  Hurricane Streets |             [Drama]|[hurricane, streets]|
|    966|Affair to Remembe...|           [Romance]|[affair, to, reme...|
|      9|   Dead Man Walking |             [Drama]|[dead, man, walking]|
|   1230|Ready to Wear Pre...|            [Comedy]|[ready, to, wear,...|
|   1118|        Up in Smoke |            [Comedy]|     [up, in, smoke]|
|    673|          Cape Fear |[Film-Noir, Thril...|        [cape, fear]|
|    879|     Peacemaker The |[Action, Thriller...|   [peacemaker, the]|
|     66|While You Were Sl...|   [Comedy, Romance]|[while, you, were...|
+-------+--------------------+--------------------+

In [27]:
# remove stop words
remover = StopWordsRemover(inputCol="title_words", outputCol="text")
clean_data = remover.transform(tokenized_data).drop("title", "title_words")
clean_data.show(10)

+-------+--------------------+--------------------+
|MovieId|              genres|                text|
+-------+--------------------+--------------------+
|    167|            [Comedy]| [private, benjamin]|
|   1343|             [Drama]|       [lotto, land]|
|   1607|             [Drama]|[hurricane, streets]|
|    966|           [Romance]|  [affair, remember]|
|      9|             [Drama]|[dead, man, walking]|
|   1230|            [Comedy]|[ready, wear, pre...|
|   1118|            [Comedy]|             [smoke]|
|    673|[Film-Noir, Thril...|        [cape, fear]|
|    879|[Action, Thriller...|        [peacemaker]|
|     66|   [Comedy, Romance]|          [sleeping]|
+-------+--------------------+--------------------+
only showing top 10 rows



### convert text input into feature vectors

In [28]:
# step 1: perform HashingTF on column "text"
text_hasher = HashingTF(inputCol="text", outputCol="text_features", numFeatures=1024)
hashed_data = text_hasher.transform(clean_data)
hashed_data.show(5)

+-------+---------+--------------------+--------------------+
|MovieId|   genres|                text|       text_features|
+-------+---------+--------------------+--------------------+
|    167| [Comedy]| [private, benjamin]|(1024,[128,544],[...|
|   1343|  [Drama]|       [lotto, land]|(1024,[38,300],[1...|
|   1607|  [Drama]|[hurricane, streets]|(1024,[592,821],[...|
|    966|[Romance]|  [affair, remember]|(1024,[389,502],[...|
|      9|  [Drama]|[dead, man, walking]|(1024,[11,342,101...|
+-------+---------+--------------------+--------------------+
only showing top 5 rows



In [29]:
# step 2: fit a CountVectorizerModel from column "genres".
count_vectorizer = CountVectorizer(inputCol="genres", outputCol="genres_features")
count_vectorizer_model = count_vectorizer.fit(hashed_data)
vectorized_data = count_vectorizer_model.transform(hashed_data)
vectorized_data.show(5)

+-------+---------+--------------------+--------------------+---------------+
|MovieId|   genres|                text|       text_features|genres_features|
+-------+---------+--------------------+--------------------+---------------+
|    167| [Comedy]| [private, benjamin]|(1024,[128,544],[...| (19,[1],[1.0])|
|   1343|  [Drama]|       [lotto, land]|(1024,[38,300],[1...| (19,[0],[1.0])|
|   1607|  [Drama]|[hurricane, streets]|(1024,[592,821],[...| (19,[0],[1.0])|
|    966|[Romance]|  [affair, remember]|(1024,[389,502],[...| (19,[4],[1.0])|
|      9|  [Drama]|[dead, man, walking]|(1024,[11,342,101...| (19,[0],[1.0])|
+-------+---------+--------------------+--------------------+---------------+
only showing top 5 rows



In [30]:
# step 3: assemble features into a single vector
assembler = VectorAssembler(
    inputCols=["text_features", "genres_features"],
    outputCol="features",
)

feature_data = assembler.transform(vectorized_data).select("MovieId", "features")

feature_data.show(10, False)

+-------+---------------------------------------------+
|MovieId|features                                     |
+-------+---------------------------------------------+
|167    |(1043,[128,544,1025],[1.0,1.0,1.0])          |
|1343   |(1043,[38,300,1024],[1.0,1.0,1.0])           |
|1607   |(1043,[592,821,1024],[1.0,1.0,1.0])          |
|966    |(1043,[389,502,1028],[1.0,1.0,1.0])          |
|9      |(1043,[11,342,1014,1024],[1.0,1.0,1.0,1.0])  |
|1230   |(1043,[597,740,902,1025],[1.0,1.0,1.0,1.0])  |
|1118   |(1043,[702,1025],[1.0,1.0])                  |
|673    |(1043,[169,690,1027,1040],[1.0,1.0,1.0,1.0]) |
|879    |(1043,[909,1026,1027,1034],[1.0,1.0,1.0,1.0])|
|66     |(1043,[256,1025,1028],[1.0,1.0,1.0])         |
+-------+---------------------------------------------+
only showing top 10 rows



The features column is represented with a SparseVector object. For example, in the feature vector (1043,[128,544,1025],[1.0,1.0,1.0]), 1043 is the vector length, indicating the vector consisting of 1043 item features. The values at index positions 128,544,1025 are 1.0, and the values at other positions are all 0.

### diversity metrics applied

In [31]:
als_eval = SparkDiversityEvaluation(
    train_df = train_df, 
    reco_df = top_k_reco,
    item_feature_df = feature_data, 
    item_sim_measure="item_feature_vector",
    col_user = COL_USER, 
    col_item = COL_ITEM
)

als_metrics = get_diversity_results(als_eval)

In [32]:
cols = ["catalog_coverage", "distributional_coverage","novelty", "diversity", "serendipity"]
content_diversity_results = pd.DataFrame(columns=cols)
content_diversity_results.loc[1] = als_metrics
content_diversity_results.head()

,catalog_coverage,distributional_coverage,novelty,diversity,serendipity
1,0.515003,8.497723,11.391092,0.871292,0.886252


In [33]:
diversity_results.head()

,catalog_coverage,distributional_coverage,novelty,diversity,serendipity
1,0.515003,8.497723,11.391092,0.880526,0.868944


## DISPLAY ALL RESULTS

In [34]:
print("ALS results for : {} of the Movielens size: {} ".format(TOP_K,MOVIELENS_DATA_SIZE))
print()

ALS results for : 20 of the Movielens size: 100k 



In [35]:
print("DIVERSITY METRICS - COLLABORATIVE")
print()
diversity_results.head()

DIVERSITY METRICS - COLLABORATIVE



,catalog_coverage,distributional_coverage,novelty,diversity,serendipity
1,0.515003,8.497723,11.391092,0.880526,0.868944


In [36]:
print("DIVERSITY METRICS - CONTENT BASED")
print()
content_diversity_results.head()

DIVERSITY METRICS - CONTENT BASED



,catalog_coverage,distributional_coverage,novelty,diversity,serendipity
1,0.515003,8.497723,11.391092,0.871292,0.886252


In [37]:
print("RATING METRICS")
print()
rating_res.head()

RATING METRICS



,rmse,mean absolute error,R squared,explained variance
1,0.971761,0.757398,0.254696,0.259558


In [38]:
print("RANKING METRICS")
print()
ranking_results.head()

RANKING METRICS



,Precision@k,Recall@k,NDCG@k,Mean average precision
1,0.053287,0.038843,0.052833,0.007391
